In [3]:
# import libraries
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# api requests
# Currently we are on congress 117


# Senator api request
Senator_response = requests.get(
    'https://api.propublica.org/congress/v1/117/senate/members.json',
    headers={'X-API-Key': 'PTmyeOwqEcHo6ZIRbr76DrAFRG649DHtwaAJAsXS'}
)
# House api request
House_response = requests.get(
    'https://api.propublica.org/congress/v1/117/house/members.json',
    headers={'X-API-Key': 'PTmyeOwqEcHo6ZIRbr76DrAFRG649DHtwaAJAsXS'}
)

#convert responses into json
senate_json_response = Senator_response.json()
house_json_response = House_response.json()

In [4]:

# Senator array
Senate = senate_json_response['results'][0]["members"]
print(len(Senate))

# House array
House = house_json_response['results'][0]["members"]    
print(len(House))

102
440


In [5]:
#Congress Array
Congress = Senate + House
print(len(Congress))

542


In [6]:
# Convert array in Dataframe
df = pd.DataFrame(Congress)

In [7]:
#dropping unnessary columns from dataframe
df = df.drop(['date_of_birth', 'twitter_account','facebook_account','youtube_account', 'rss_url','contact_form', 'in_office', 'ideal_point', 'contact_form','next_election',
'total_votes', 'missed_votes', 'total_present', 'last_updated', 'office', 'phone', 'fax', 'at_large', 'geoid', 'cook_pvi','dw_nominate', 'votes_with_party_pct', 'votes_with_party_pct', 'missed_votes_pct','votes_against_party_pct'], axis=1)

In [8]:
df.keys()

Index(['id', 'title', 'short_title', 'api_uri', 'first_name', 'middle_name',
       'last_name', 'suffix', 'gender', 'party', 'leadership_role',
       'govtrack_id', 'cspan_id', 'votesmart_id', 'icpsr_id', 'crp_id',
       'google_entity_id', 'fec_candidate_id', 'url', 'seniority', 'ocd_id',
       'state', 'senate_class', 'state_rank', 'lis_id', 'district'],
      dtype='object')

In [42]:
# Column that we are adding to the dataframe
df['image'] = "None"
# VTO20E = Voted to overturn 2020 election
df['VTO20E'] = False
# VTITAI = Voted to impeach Trump after inserection
df['VTITAI'] = 1.0   

In [10]:
# States for scraping
states = ["alabama", 'alaska',"arizona", "arkansas", "california", "colorado",'connecticut','delaware', 'florida','georgia', 'hawaii','idaho', 'illinois', 'indiana', 'iowa', 'kansas', 'kentucky', 'louisiana', 'maine', 'maryland', 'massachusetts', 'michigan', 'minnesota', 'mississippi', 'missouri','montana', 'nebraska','nevada', 'new%20hampshire', 'new%20jersey','new%20mexico', 'new%20york', 'north%20carolina', 'north%20dakota', 'ohio', 'oklahoma', 'oregon', 'pennsylvania','puerto%20rico', 'rhode%20island', 'south%20carolina', 'south%20dakota', 'tennessee', 'texas', 'utah', 'vermont', 'virginia', 'washington', 'west%20virginia', 'wisconsin', 'wyoming' ]

Main_url = 'https://www.congress.gov/search?searchResultViewType=expanded&q={%22congress%22:[%22116%22],%22source%22:[%22members%22],%22search%22:%22'
back_of_Main_url = "%22}"
top_images_url = 'https://congress.gov'
#reps with no photos on Congress.gov
no_photos = []
Congress_dic = {}
Congress_dic['congress_info'] = []

In [11]:
# Checking to see if there is a picture on the website
def CheckForImg(str):
    try:
        rep_Img = rep.find("img")['src']
        return True
    except:
        return False

In [12]:
# State Abbrevation Map Key
state_abrv = {"alabama":'AL', 'alaska':'AL',"arizona":'AZ', "arkansas":'AK', "california":'CA', "colorado":'CO','connecticut':'CT','delaware':'DE', 'florida':'FL','georgia':'GA', 'hawaii':'HI','idaho':'ID', 'illinois':'IL', 'indiana':'ID', 'iowa':'IA', 'kansas':'KS', 'kentucky':'KY', 'louisiana':'LA', 'maine':'ME', 'maryland':'MD', 'massachusetts':'MA', 'michigan':"MI", 'minnesota':'MN', 'mississippi':'MS', 'missouri':'MS','montana':'MT', 'nebraska':'NE','nevada':'NV', 'new%20hampshire':'NH', 'new%20jersey':'NJ','new%20mexico':'NM', 'new%20york':'NY', 'north%20carolina':'NC', 'north%20dakota':'ND', 'ohio':'OH', 'oklahoma':'OK', 'oregon':'OR', 'pennsylvania':'PA','puerto%20rico':'PR','rhode%20island':'RI', 'south%20carolina':'SC', 'south%20dakota':'SD', 'tennessee':'TN', 'texas':'TX', 'utah':'UT', 'vermont':'VT', 'virginia':'VA', 'washington':'WA', 'west%20virginia':'WV', 'wisconsin':'WI', 'wyoming':'WY'}

In [50]:
# # for loop that is going thru every state
for state in states: 
    state_res = requests.get(Main_url+state+back_of_Main_url)
    state_soup = BeautifulSoup(state_res.text, 'html.parser')
    Rep_results = state_soup.findAll("li", {"class":"expanded"})

    for rep in Rep_results:
        photo = CheckForImg(rep)
        rep_name = rep.find("span", {"class":"result-heading"}).text.split(' ')[1:]
        rep_index = df.loc[(df['state'] == state_abrv[state]) & (df['last_name'] == rep_name[0][:-1])].index
        if photo:
            df.at[rep_index, 'image'] = rep_Img = rep.find("img")['src']

In [43]:
# Switching all House and Senate Republicans that they didnt want to impeach trump
df.loc[(df.party == "R"), ['VTITAI']] = 0.0

In [46]:
# House Members who didnt appear to vote on impeachmenting charges
no_vote = ["Kay Granger", "Andy Harris", "Greg Murphy", "Daniel Webster"]
for pussy in no_vote:
    name = pussy.split(" ")
    rep_index = df.loc[(df['party'] == "R") & (df['last_name'] == name[1]) & (df['first_name'] == name[0])].index
    df.at[rep_index, 'VTITAI'] = None

In [44]:
# Republican House & Senate members who voted to impeach Trump
RepHouseHeros = ["Liz, Cheney", "Anthony, Gonzalez", "Jaime, Herrera Beutler", "John, Katko", "Adam, Kininger", "Peter, Meijer", "Dan, Newhouse", "Tom, Rice", "Fred, Upton", "David, Valadao", "Susan, Collins", "Lisa, Murkowski", "Mitt, Romney", "Ben, Sasse", " Bill, Cassidy", "Patrick, Toomey", "Richard, Burr"]

for hero in RepHouseHeros:
    name = hero.split(", ")
    rep_index = df.loc[(df['party'] == "R") & (df['last_name'] == name[1]) & (df['first_name'] == name[0])].index
    df.at[rep_index, 'VTITAI'] = 1.0

In [69]:
# idiots who voted to overturn 2020 election
idiots = ["Tommy, Tuberville", "Rick, Scott", "Roger, Marshall", "John, Kennedy", "Cindy, Hyde-Smith", "Josh, Hawley", "Ted, Cruz", "Cynthia, Lummis", "Robert, Aderholt", "Mo, Brooks", "Jerry, Carl", "Barry, Moore", "Gary, Palmer", "Mike, Rogers", "Andy, Biggs", "Paul, Gosar", "Debbie, Lesko", "David, Schweikert", "Rick, Crawford", "Ken, Calvert", "Mike, Garcia", "Darrell, Issa", "Doug, LaMalfa", "Kevin, McCarthy", "Devin, Nunes", "Jay, Obernolte", "Lauren, Boebert", "Doug, Lamborn", "Kat, Cammack", "Mario, Diaz-Balart", "Byron, Donalds", "Neal, Dunn", "Scott, Franklin", "Matt, Gaetz", "Carlos, Gimenez", "Brian, Mast", "Bill, Posey", "John, Rutherford", "Greg, Steube", "Daniel, Webster", "Rick, Allen", "Buddy, Carter", "Andrew, Clyde","Marjorie, Greene", "Jody, Hice", "Barry, LouderMilk", "Russ, Fulcher", "Mike, Bost", "Mary, Miller", "Jim, Barid", "Jim, Banks", "Greg, Pence", "Jackie, Walorski", "Ron, Estes", "Jacob, LaTurner", "Tracey, Mann", "Harold, Rogers", "Garret, Greaves", "Clay, Higgins", "Mike, Johnson", "Steve, Scalise", "Andy, Harris", "Jack, Bergman", "Lisa, McClain", "Tim, Walberg", "Michelle, Fischbach", "Jim, Hagedorn", "Michael, Guest", "Trent, Kelly", "Steven, Palazzo", "Sam, Graves", "Vicky, Hartzler", "Billy, Long", "Blaine, Luetkemeyer", "Jason, Smith", "Matt, Rosendale", "Dan, Bishop", "Ted, Budd", "Madison, Cawthorn", "Virginia, Foxx", "Richard, Hudson", "Greg, Murphy", "David, Rouzer", "Jeff, Van Drew" , "Yvette, Herrell", "Chris, Jacobs", "Nicole, Malliotakis", "Elise, Stefanik", "Lee, Zeldin", "Adrian, Smith", "Steve, Chabot", "Warren, Davidson", "Bob, Gibbs", "Bill, Johnson", "Jim, Jordan", "Stephaine, Bice", "Tom, Cole", "Kevin, Hern", "Frank, Lucas", "Markwayne, Mullin", "Cliff, Bentz", "John, Joyce", "Fred, Keller", "Mike, Kelly", "Daniel, Meuser", "Scott, Perry", "Guy, Reschenthaler", "Lloyd, Smucker", "Glenn, Thompson", "Jeff, Duncan", "Ralph, Norman", "Tom, Rice", "William, Timmons", "Joe, Wilson", "Tim, Burchett", "Scott, DesJarlais", "Chuck, Fleischmann", "Mark, Green", "Diana, Harshbarger", "David, Kustoff", "John, Rose", "Jodey, Arrington", "Brian, Babin", "Michael, Burgess", "John, Carter", "Michael, Cloud", "Pat, Fallon", "Louie, Gohmert", "Lance, Gooden", "Ronny, Jackson", "Troy, Nehls", "August, Pfluger", "Pete, Sessions", "Beth, Van Duyne", "Randy, Weber", "Roger, Williams", "Ron, Wright", "Burgess, Owens", "Chris, Stewart", "Ben, Cline", "Bob, Good", "Morgan, Griffith", "Robert, Wittman", "Carol, Miller", "Alex, Mooney", "Scott, Fitzgerald", "Tom, Tiffany"]

In [72]:
for idiot in idiots:
    name = idiot.split(", ")
    rep_index = df.loc[(df['party'] == "R") & (df['first_name'] == name[0]) & (df['last_name'] == name[1])].index
    df.at[rep_index, 'VTO20E'] = True

In [75]:
# export Dataframe to CVS
df.to_csv('CongressData.csv', index=False)